# Memory System Practice

This notebook demonstrates the complete **three-tier memory system** implementation.

## Memory Architecture

The system implements:

1. **Working Memory (WM)**: Recent conversation turns (in-memory, last N messages)
2. **Episodic Memory (EM)**: Session-specific facts distilled from conversations
3. **Semantic Memory (SM)**: Long-term knowledge from documents

## Contents
1. Setup and Initialization
2. Working Memory (Session Management)
3. Intent Classification and Entity Extraction
4. Episodic Memory Operations
5. Semantic Memory Retrieval
6. Combined Memory Recall (EM + SM)
7. LangGraph State Machine Flow
8. Salience Tracking and Distillation
9. End-to-End Scenarios

## 1. Setup and Initialization

In [1]:
# Add project root to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import memory system components
from acc_llamaindex.config import config
from acc_llamaindex.application.chat_service.session_manager import session_manager
from acc_llamaindex.application.memory_service.intent_classifier import intent_classifier
from acc_llamaindex.application.memory_service.entity_extractor import entity_extractor
from acc_llamaindex.application.memory_service.service import memory_service
from acc_llamaindex.application.memory_service.salience_tracker import salience_tracker
from acc_llamaindex.application.memory_service.conversation_summarizer import conversation_summarizer
from acc_llamaindex.application.chat_service.graph import get_memory_graph, MemoryState
from acc_llamaindex.infrastructure.llm_providers.langchain_provider import get_llm, reset_llm
from acc_llamaindex.infrastructure.db.chroma_client import chroma_client

print("✓ All memory system imports successful")

/Users/kevinknox/coding/acc-llamaindex/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-19 21:48:31.201 | INFO     | acc_llamaindex.application.reranking_service.service:__init__:18 - RerankingService initialized
2025-10-19 21:48:31.213 | INFO     | acc_llamaindex.application.evaluation_service.service:__init__:15 - EvaluationService initialized
2025-10-19 21:48:31.213 | INFO     | acc_llamaindex.application.chat_service.service:__init__:20 - ChatService initialized
2025-10-19 21:48:31.214 | INFO     | acc_llamaindex.application.chat_service.session_manager:__init__:25 - SessionManager initialized (max_sessions=100, ttl=30min)
2025-10-19 21:48:31.215 | INFO     | acc_llamaindex.application.memory_service.service:__init__:23 - MemoryService initialized
2025-10-19 21:48:31.217 | INFO     | ac

✓ All memory system imports successful


## 2. Working Memory (Session Management)

Working Memory stores recent conversation turns in-memory with:
- LRU eviction when max sessions reached
- TTL-based cleanup of expired sessions
- Thread-safe operations

### Pattern 1: Create and Manage Sessions

In [2]:
# Initialize core services
chroma_client.initialize()
get_llm()

print(f"\nMemory System Config:")
print(f"- WM Max Turns: {config.wm_max_turns}")
print(f"- WM Max Sessions: {config.wm_max_sessions}")
print(f"- WM Session TTL: {config.wm_session_ttl_minutes} minutes")
print(f"- EM Collection: {config.em_collection_name}")
print(f"- SM Collection: {config.chroma_collection_name}")  # SM uses main collection
print(f"- EM K Default: {config.em_k_default}")
print(f"- SM K Default: {config.sm_k_default}")
print(f"- Enable Memory System: {config.enable_memory_system}")
print(f"- Enable EM Distillation: {config.enable_em_distillation}")
print(f"- Distill Every N Turns: {config.em_distill_every_n_turns}")

2025-10-19 21:48:32.955 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:initialize:34 - Initializing ChromaDB at /Users/kevinknox/coding/acc-llamaindex/data/chroma_db
2025-10-19 21:48:33.023 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:initialize:42 - ChromaDB client and embeddings initialized
2025-10-19 21:48:33.030 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:initialize:55 - Semantic memory collection initialized: documents
2025-10-19 21:48:33.030 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:_initialize_episodic_memory:73 - Initializing episodic memory collection: episodic_memory
2025-10-19 21:48:33.031 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:_initialize_episodic_memory:86 - Episodic memory collection initialized successfully



Memory System Config:
- WM Max Turns: 10
- WM Max Sessions: 100
- WM Session TTL: 30 minutes
- EM Collection: episodic_memory
- SM Collection: documents
- EM K Default: 5
- SM K Default: 10
- Enable Memory System: True
- Enable EM Distillation: True
- Distill Every N Turns: 5


In [3]:
# Create a new session
session_id = "demo-session-001"
session = session_manager.create_session(session_id)

print(f"Created session: {session['session_id']}")
print(f"Created at: {session['created_at']}")
print(f"Turn count: {session['turn_count']}")

2025-10-19 21:48:37.192 | INFO     | acc_llamaindex.application.chat_service.session_manager:create_session:88 - Created session demo-session-001


Created session: demo-session-001
Created at: 2025-10-19 21:48:37.192629
Turn count: 0


### Pattern 2: Add Conversation Turns

In [4]:
# Add conversation turns
session_manager.add_turn(
    session_id=session_id,
    user_message="What is the African Growth and Opportunity Act?",
    assistant_message="AGOA is a U.S. trade program providing preferential access to African countries.",
    metadata={"intent": "compliance_query", "entities": [{"text": "AGOA", "type": "program"}]}
)

session_manager.add_turn(
    session_id=session_id,
    user_message="Which countries are eligible?",
    assistant_message="Sub-Saharan African countries that meet eligibility criteria.",
    metadata={"intent": "general", "entities": []}
)

print(f"Added 2 turns to session {session_id}")

2025-10-19 21:48:39.334 | DEBUG    | acc_llamaindex.application.chat_service.session_manager:add_turn:134 - Added turn 1 to session demo-session-001
2025-10-19 21:48:39.335 | DEBUG    | acc_llamaindex.application.chat_service.session_manager:add_turn:134 - Added turn 2 to session demo-session-001


Added 2 turns to session demo-session-001


### Pattern 3: Retrieve Recent Turns

In [5]:
# Get recent turns
recent_turns = session_manager.get_recent_turns(session_id, n=2)

print(f"Retrieved {len(recent_turns)} recent turns:\n")
for turn in recent_turns:
    print(f"Turn {turn['turn_number']}:")
    print(f"  User: {turn['user_message']}")
    print(f"  Assistant: {turn['assistant_message']}")
    print(f"  Metadata: {turn['metadata']}\n")

Retrieved 2 recent turns:

Turn 1:
  User: What is the African Growth and Opportunity Act?
  Assistant: AGOA is a U.S. trade program providing preferential access to African countries.
  Metadata: {'intent': 'compliance_query', 'entities': [{'text': 'AGOA', 'type': 'program'}]}

Turn 2:
  User: Which countries are eligible?
  Assistant: Sub-Saharan African countries that meet eligibility criteria.
  Metadata: {'intent': 'general', 'entities': []}



## 3. Intent Classification and Entity Extraction

Intent classification helps route queries:
- `quote_request`: Price quotes
- `compliance_query`: Regulations
- `shipment_tracking`: Tracking
- `general`: General questions

### Pattern 1: Classify User Intents

In [6]:
# Test queries with different intents
test_queries = [
    "What is the price for shipping 100 units to Lagos?",
    "What are the import regulations for textiles?",
    "Where is my shipment TRK-12345?",
    "What is AGOA?"
]

print("Intent Classification Results:\n")
for query in test_queries:
    result = intent_classifier.classify(query)
    print(f"Query: {query}")
    print(f"  Intent: {result['intent']} (confidence: {result['confidence']:.2f})")
    print(f"  Reasoning: {result.get('reasoning', 'N/A')}\n")

2025-10-19 21:48:45.572 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:66 - Classifying intent for query: What is the price for shipping 100 units to Lagos?


Intent Classification Results:



2025-10-19 21:48:48.392 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:88 - Intent classified: quote_request (confidence: 0.85)
2025-10-19 21:48:48.393 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:66 - Classifying intent for query: What are the import regulations for textiles?


Query: What is the price for shipping 100 units to Lagos?
  Intent: quote_request (confidence: 0.85)
  Reasoning: User asked for the price/quote for shipping 100 units to Lagos, which is a request for a shipping quote.



2025-10-19 21:48:50.211 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:88 - Intent classified: compliance_query (confidence: 0.80)
2025-10-19 21:48:50.212 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:66 - Classifying intent for query: Where is my shipment TRK-12345?


Query: What are the import regulations for textiles?
  Intent: compliance_query (confidence: 0.80)
  Reasoning: User asked about import regulations for textiles, which relates to regulatory/compliance information (customs, documentation, standards).



2025-10-19 21:48:53.508 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:88 - Intent classified: shipment_tracking (confidence: 0.92)
2025-10-19 21:48:53.508 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:66 - Classifying intent for query: What is AGOA?


Query: Where is my shipment TRK-12345?
  Intent: shipment_tracking (confidence: 0.92)
  Reasoning: User asks for the current location/status of a shipment using a tracking number TRK-12345, which is a tracking inquiry.



2025-10-19 21:48:55.905 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:88 - Intent classified: compliance_query (confidence: 0.78)


Query: What is AGOA?
  Intent: compliance_query (confidence: 0.78)
  Reasoning: AGOA is a trade regulation/program (African Growth and Opportunity Act) relevant to shipments and compliance requirements.



### Pattern 2: Extract Entities

In [7]:
# Extract entities from queries
query = "I need to ship 500 smartphones from China to Nigeria using FOB Incoterms"
entities = entity_extractor.extract(query)

print(f"Query: {query}\n")
print(f"Extracted Entities:")
for entity in entities:
    print(f"  - {entity['text']} ({entity['type']})")

2025-10-19 21:48:56.821 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:_extract_with_llm:191 - Using LLM fallback for entity extraction: I need to ship 500 smartphones from China to Niger
2025-10-19 21:49:05.626 | INFO     | acc_llamaindex.application.memory_service.entity_extractor:extract:137 - Extracted 4 entities from query: I need to ship 500 smartphones from China to Niger
2025-10-19 21:49:05.626 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:extract:139 -   - incoterm: FOB (incoterm:fob)
2025-10-19 21:49:05.627 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:extract:139 -   - commodity: smartphones (commodity:smartphones)
2025-10-19 21:49:05.627 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:extract:139 -   - country: China (country:china)
2025-10-19 21:49:05.628 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:extract:139 -   - country: Nigeria (country:nige

Query: I need to ship 500 smartphones from China to Nigeria using FOB Incoterms

Extracted Entities:
  - FOB (incoterm)
  - smartphones (commodity)
  - China (country)
  - Nigeria (country)


## 4. Episodic Memory Operations

Episodic Memory stores session-specific facts extracted from conversations.

### Pattern 1: Add Facts to Episodic Memory

In [8]:
# Add episodic facts manually (normally done via distillation)
facts = [
    "User is interested in exporting textiles to Nigeria",
    "User prefers FOB Incoterms for shipments",
    "User mentioned working with supplier in Lagos"
]

for i, fact in enumerate(facts, 1):
    chroma_client.write_episodic(
        texts=[fact],
        metadatas=[{  # Must be a list of dicts
            "session_id": session_id,
            "fact_id": f"fact-{i}",
            "source": "manual_test",
            "salience_score": 0.7
        }]
    )
    print(f"Added EM fact {i}: {fact}")

2025-10-19 21:49:10.138 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:write_episodic:202 - Wrote 1 items to episodic memory
2025-10-19 21:49:10.296 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:write_episodic:202 - Wrote 1 items to episodic memory


Added EM fact 1: User is interested in exporting textiles to Nigeria
Added EM fact 2: User prefers FOB Incoterms for shipments


2025-10-19 21:49:10.451 | INFO     | acc_llamaindex.infrastructure.db.chroma_client:write_episodic:202 - Wrote 1 items to episodic memory


Added EM fact 3: User mentioned working with supplier in Lagos


### Pattern 2: Query Episodic Memory

In [9]:
# Query EM for session-specific context
query = "What shipping terms does the user prefer?"

em_results = memory_service._query_episodic(
    session_id=session_id,
    query=query,
    k=3
)

print(f"Query: {query}\n")
print(f"Episodic Memory Results ({len(em_results)} found):\n")
for i, result in enumerate(em_results, 1):
    print(f"Result {i}:")
    print(f"  Text: {result['text']}")
    print(f"  Salience: {result.get('salience', 0.0):.4f}")
    print(f"  Source: {result.get('source', 'unknown')}")
    print(f"  Metadata: {result.get('metadata', {})}\n")

2025-10-19 21:49:19.052 | DEBUG    | acc_llamaindex.application.memory_service.service:_query_episodic:110 - Retrieved 3 EM results for session demo-session-001


Query: What shipping terms does the user prefer?

Episodic Memory Results (3 found):

Result 1:
  Text: User prefers FOB Incoterms for shipments
  Salience: 0.5000
  Source: EM
  Metadata: {'source': 'manual_test', 'salience_score': 0.7, 'fact_id': 'fact-2', 'session_id': 'demo-session-001'}

Result 2:
  Text: User prefers FOB Incoterms for shipments
  Salience: 0.5000
  Source: EM
  Metadata: {'session_id': 'demo-session-001', 'fact_id': 'fact-2', 'source': 'manual_test', 'salience_score': 0.7}

Result 3:
  Text: User is interested in exporting textiles to Nigeria
  Salience: 0.5000
  Source: EM
  Metadata: {'fact_id': 'fact-1', 'salience_score': 0.7, 'source': 'manual_test', 'session_id': 'demo-session-001'}



## 5. Semantic Memory Retrieval

Semantic Memory contains long-term knowledge from documents.

### Pattern 1: Query Semantic Memory

In [10]:
# Query SM for document knowledge
query = "What are FOB Incoterms?"

sm_results = memory_service.recall(
    query=query,
    session_id="any-session",
    intent="general",
    entities=[{"text": "FOB", "type": "incoterm"}],
    k_em=0,  # Only SM
    k_sm=5
)

print(f"Query: {query}\n")
print(f"Semantic Memory Results ({len(sm_results['sm_results'])} found):\n")
for i, result in enumerate(sm_results['sm_results'], 1):
    print(f"Result {i}:")
    print(f"  Text: {result['text'][:200]}...")
    print(f"  salience: {result['salience']:.4f}")
    print(f"  Source: {result.get('source', 'unknown')}\n")

2025-10-19 21:49:26.063 | INFO     | acc_llamaindex.application.memory_service.service:recall:60 - Memory recall: query=What are FOB Incoterms?, intent=general, session=any-session, entities=1
2025-10-19 21:49:27.347 | DEBUG    | acc_llamaindex.application.memory_service.service:_query_episodic:110 - Retrieved 0 EM results for session any-session
2025-10-19 21:49:27.543 | DEBUG    | acc_llamaindex.application.memory_service.service:_query_semantic:162 - Retrieved 5 SM results for intent=general, entities=1
2025-10-19 21:49:27.543 | DEBUG    | acc_llamaindex.application.memory_service.service:_merge_results:201 - Merged results: 0 EM + 5 SM = 5 (deduped)


Query: What are FOB Incoterms?

Semantic Memory Results (5 found):

Result 1:
  Text: FAS - Free Alongside Ship
“Free Alongside Ship” means that the seller delivers when the goods are placed alongside the vessel (e.g., on a
quay or a barge) nominated by the buyer at the named port of s...
  salience: 0.0000
  Source: SM

Result 2:
  Text: If selling on FOB terms:
You will only have to cover the costs to get the goods loaded on board the vessel ready for export – so you
will cover the container trucking from your warehouse to the port p...
  salience: 0.0000
  Source: SM

Result 3:
  Text: Put simply, Incoterms® are the selling terms that the buyer and seller of goods both agrees to.  The Incoterm®
clearly states which tasks, costs and risks are associated with the buyer and the seller....
  salience: 0.0000
  Source: SM

Result 4:
  Text: Buyer’s and Seller’s Own Transport
Under Incoterms® 2010 it was assumed that all transport would be undertaken by a third party transport
provider. U

## 6. Combined Memory Recall (EM + SM)

The memory service intelligently combines episodic and semantic memory.

### Pattern 1: Basic Combined Recall

In [11]:
# Combined EM + SM recall
query = "What Incoterms should I use for my Nigeria shipment?"

combined_results = memory_service.recall(
    query=query,
    session_id=session_id,
    intent="general",
    entities=[{"text": "Nigeria", "type": "location"}, {"text": "Incoterms", "type": "term"}],
    k_em=3,
    k_sm=3
)

print(f"Query: {query}\n")
print(f"Episodic Memory ({len(combined_results['em_results'])} results):") 
for result in combined_results['em_results']:
    print(f"  - {result['text']}")

print(f"\nSemantic Memory ({len(combined_results['sm_results'])} results):")
for result in combined_results['sm_results']:
    print(f"  - {result['text'][:100]}...")

print(f"\nCombined & Reranked ({len(combined_results['combined_results'])} results):")
for i, result in enumerate(combined_results['combined_results'], 1):
    source = result.get('source', 'unknown')
    print(f"  {i}. [{source}] {result['text'][:80]}...")

2025-10-19 21:49:31.450 | INFO     | acc_llamaindex.application.memory_service.service:recall:60 - Memory recall: query=What Incoterms should I use for my Nigeria shipmen, intent=general, session=demo-session-001, entities=2
2025-10-19 21:49:31.647 | DEBUG    | acc_llamaindex.application.memory_service.service:_query_episodic:110 - Retrieved 3 EM results for session demo-session-001
2025-10-19 21:49:31.861 | DEBUG    | acc_llamaindex.application.memory_service.service:_query_semantic:162 - Retrieved 3 SM results for intent=general, entities=2
2025-10-19 21:49:31.861 | DEBUG    | acc_llamaindex.application.memory_service.service:_merge_results:201 - Merged results: 3 EM + 3 SM = 5 (deduped)


Query: What Incoterms should I use for my Nigeria shipment?

Episodic Memory (3 results):
  - User prefers FOB Incoterms for shipments
  - User prefers FOB Incoterms for shipments
  - User is interested in exporting textiles to Nigeria

Semantic Memory (3 results):
  - to, which may occur when it is being used to confirm complex commercial agreements.
All parties must...
  - Letter of Credit.
Therefore provisions have been made to the Incoterms® 2020 to state that the buyer...
  - Ensure that you make changes to any contracts and documents as necessary
Ensure that you are stating...

Combined & Reranked (5 results):
  1. [EM] User prefers FOB Incoterms for shipments...
  2. [EM] User is interested in exporting textiles to Nigeria...
  3. [SM] to, which may occur when it is being used to confirm complex commercial agreemen...
  4. [SM] Letter of Credit.
Therefore provisions have been made to the Incoterms® 2020 to ...
  5. [SM] Ensure that you make changes to any contracts and documents

## 7. LangGraph State Machine Flow

The memory graph orchestrates the entire memory flow.

### Pattern 1: Execute Full Memory Graph

In [12]:
# Get the compiled memory graph
memory_graph = get_memory_graph()

# Create initial state
initial_state = MemoryState(
    user_query="What Incoterms are best for shipping to Africa?",
    session_id=session_id,
    wm_context=[],
    intent="",
    confidence=0.0,
    entities=[],
    em_results=[],
    sm_results=[],
    reranked_context=[],
    response="",
    citations=[],
    should_distill=False,
    retrieval_ms=0.0,
    generation_ms=0.0
)

# Execute the graph
print("Executing memory graph...\n")
final_state = memory_graph.invoke(initial_state)

print("\n=== Memory Graph Execution Complete ===\n")
print(f"User Query: {final_state['user_query']}")
print(f"\nIntent: {final_state['intent']} (confidence: {final_state['confidence']:.2f})")
print(f"Entities: {[e.get('text', '') for e in final_state['entities']]}")
print(f"\nWorking Memory: {len(final_state['wm_context'])} turns")
print(f"EM Results: {len(final_state['em_results'])}")
print(f"SM Results: {len(final_state['sm_results'])}")
print(f"Reranked Context: {len(final_state['reranked_context'])}")
print(f"\nResponse:\n{final_state['response']}")
print(f"\nCitations: {len(final_state['citations'])}")
print(f"Should Distill: {final_state['should_distill']}")

2025-10-19 21:49:38.014 | INFO     | acc_llamaindex.application.chat_service.graph:create_memory_graph:343 - Memory graph created successfully
2025-10-19 21:49:38.022 | INFO     | acc_llamaindex.application.chat_service.graph:load_working_memory:58 - Loading working memory for session: demo-session-001
2025-10-19 21:49:38.023 | DEBUG    | acc_llamaindex.application.chat_service.graph:load_working_memory:67 - Loaded 2 turns from working memory
2025-10-19 21:49:38.024 | INFO     | acc_llamaindex.application.chat_service.graph:classify_intent:82 - Classifying intent for query: What Incoterms are best for shipping to Africa?...
2025-10-19 21:49:38.024 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:66 - Classifying intent for query: What Incoterms are best for shipping to Africa?


Executing memory graph...



2025-10-19 21:49:41.510 | INFO     | acc_llamaindex.application.memory_service.intent_classifier:_classify_cached:88 - Intent classified: compliance_query (confidence: 0.62)
2025-10-19 21:49:41.511 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:_extract_with_llm:191 - Using LLM fallback for entity extraction: What Incoterms are best for shipping to Africa?
2025-10-19 21:49:50.360 | INFO     | acc_llamaindex.application.memory_service.entity_extractor:extract:137 - Extracted 1 entities from query: What Incoterms are best for shipping to Africa?
2025-10-19 21:49:50.360 | DEBUG    | acc_llamaindex.application.memory_service.entity_extractor:extract:139 -   - incoterm: Incoterms (incoterm:incoterms)
2025-10-19 21:49:50.360 | INFO     | acc_llamaindex.application.chat_service.graph:classify_intent:97 - Intent: compliance_query (confidence: 0.62), Entities: 1
2025-10-19 21:49:50.362 | INFO     | acc_llamaindex.application.chat_service.graph:query_episodic_memory:115 


=== Memory Graph Execution Complete ===

User Query: What Incoterms are best for shipping to Africa?

Intent: compliance_query (confidence: 0.62)
Entities: ['Incoterms']

Working Memory: 2 turns
EM Results: 0
SM Results: 0
Reranked Context: 3

Response:
Based on your context (you prefer FOB, you’re exporting textiles to Nigeria, and you’re working with a Lagos supplier), here are the Incoterms that tend to work best for shipping to Africa, with notes on when to use them:

- FOB (Free On Board) — strongest default for sea shipments from Lagos
  - Your Lagos supplier handles export clearance and loads the goods onto the ship.
  - Risk transfers to you when the goods pass the ship’s rail; you then arrange ocean freight and insurance from the port onward.
  - This matches your stated preference for FOB [EM-1] and fits your Lagos supplier setup [EM-3].

- CFR (Cost and Freight) — if you want the seller to pay freight to the destination port, but you handle insurance
  - Seller covers cost 

## 8. Salience Tracking and Distillation

Salience tracking determines which memories are most important.

### Pattern 1: Track Citation Salience

In [ ]:
# Simulate tracking citations
cited_facts = [
    {"text": "User prefers FOB Incoterms", "id": "fact-1"},
    {"text": "User is exporting textiles", "id": "fact-2"}
]

salience_tracker.track_citations(cited_facts)
print("Tracked citations for salience scoring")

# Get salience scores
for fact in cited_facts:
    score = salience_tracker.get_salience_score(fact['id'])
    print(f"  {fact['text']}: {score:.4f}")

AttributeError: 'dict' object has no attribute 'llm_provider'

### Pattern 2: Conversation Distillation

In [ ]:
# Get recent turns to distill
turns_to_distill = session_manager.get_recent_turns(session_id, n=2)

print(f"Distilling {len(turns_to_distill)} conversation turns...\n")

# Distill into facts
distill_result = conversation_summarizer.distill(
    session_id=session_id,
    turns=turns_to_distill
)

print(f"Distillation Result:")
print(f"  Success: {distill_result.get('success', False)}")
print(f"  Facts Created: {distill_result.get('facts_created', 0)}")
print(f"\nExtracted Facts:")
for fact in distill_result.get('facts', []):
    print(f"  - {fact}")

Agent handled potential errors gracefully


## 9. End-to-End Scenario

Complete workflow demonstrating the full memory system.

### Scenario: Multi-Turn Conversation with Memory

In [ ]:
# Create a new session for this scenario
scenario_session_id = "scenario-001"

print("Simulating Multi-Turn Conversation:\n")

# Turn 1
state1 = MemoryState(
    user_query="What are Incoterms?",
    session_id=scenario_session_id,
    wm_context=[], intent="", confidence=0.0, entities=[],
    em_results=[], sm_results=[], reranked_context=[],
    response="", citations=[], should_distill=False,
    retrieval_ms=0.0, generation_ms=0.0
)
result1 = memory_graph.invoke(state1)
print(f"Turn 1:")
print(f"  User: {result1['user_query']}")
print(f"  Intent: {result1['intent']}")
print(f"  Response: {result1['response'][:100]}...\n")

# Turn 2
state2 = MemoryState(
    user_query="Which one is best for sea freight?",
    session_id=scenario_session_id,
    wm_context=[], intent="", confidence=0.0, entities=[],
    em_results=[], sm_results=[], reranked_context=[],
    response="", citations=[], should_distill=False,
    retrieval_ms=0.0, generation_ms=0.0
)
result2 = memory_graph.invoke(state2)
print(f"Turn 2:")
print(f"  User: {result2['user_query']}")
print(f"  WM Context: {len(result2['wm_context'])} previous turns")
print(f"  Response: {result2['response'][:100]}...\n")

# Turn 3 - tests memory recall
state3 = MemoryState(
    user_query="What did we discuss earlier?",
    session_id=scenario_session_id,
    wm_context=[], intent="", confidence=0.0, entities=[],
    em_results=[], sm_results=[], reranked_context=[],
    response="", citations=[], should_distill=False,
    retrieval_ms=0.0, generation_ms=0.0
)
result3 = memory_graph.invoke(state3)
print(f"Turn 3:")
print(f"  User: {result3['user_query']}")
print(f"  WM Context: {len(result3['wm_context'])} previous turns")
print(f"  EM Results: {len(result3['em_results'])}")
print(f"  Response: {result3['response'][:150]}...\n")

print("\n=== Scenario Complete ===")
session_stats = session_manager.get_session(scenario_session_id)
print(f"Total turns in session: {session_stats['turn_count']}")

## Summary

This notebook demonstrated:

1. **Working Memory**: In-memory session management with LRU eviction and TTL cleanup
2. **Intent Classification**: LLM-based routing for different query types
3. **Entity Extraction**: Identifying key entities in user queries
4. **Episodic Memory**: Session-specific facts from conversation distillation
5. **Semantic Memory**: Long-term document knowledge retrieval
6. **Memory Fusion**: Combining EM + SM with intelligent reranking
7. **LangGraph Flow**: State machine orchestration of all memory tiers
8. **Salience Tracking**: Importance scoring for memory items
9. **Distillation**: Automatic fact extraction from conversations
10. **End-to-End**: Full multi-turn conversations with memory

## Key Takeaways

- **WM** provides fast access to recent context
- **EM** stores personalized session facts
- **SM** retrieves relevant document knowledge
- **LangGraph** coordinates the flow between memory tiers
- **Salience** ensures important facts are retained
- **Distillation** automatically extracts conversation facts

## Next Steps

- Experiment with different distillation frequencies
- Test salience-based memory promotion
- Implement custom reranking strategies
- Add memory decay mechanisms
- Explore cross-session memory sharing